<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1aWSFb-ZR8TTIDdRlDBBCh-YvvCxmt6Bc

Retrieving folder contents
Processing file 1ppSSIuo0o740Z-Wfz6m47xQtmHd08znO bitsandbytes-0.43.0.dev0-cp311-cp311-win_amd64.whl
Processing file 1I6I0pdL4QNcozYaWng7i1GnmPmqwWJZC deepspeed-0.13.1+unknown-py3-none-any.whl
Processing file 1xU2jfkljiVuqqQjCkYPVa1KUFR8QF_SB llvm-5e5a22ca-windows-x64.tar.gz
Processing file 1Xt_XGgc_wgl1f079oSjDNfCBhd7iXnzC triton-2.1.0-cp311-cp311-win_amd64.whl
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1ppSSIuo0o740Z-Wfz6m47xQtmHd08znO
To: /content/311Wheels/bitsandbytes-0.43.0.dev0-cp311-cp311-win_amd64.whl
100% 24.1M/24.1M [00:00<00:00, 60.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1I6I0pdL4QNcozYaWng7i1GnmPmqwWJZC
To: /content/311Wheels/deepspeed-0.13.1+unknown-py3-none-any.whl
100% 961k/961k [00:00<00:00, 129MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1xU2jfkljiVuqqQjCkYPVa1KUFR8QF_SB
From (redirected

In [ ]:
from google.colab import files
files.download('/content/311Wheels/llvm-5e5a22ca-windows-x64.tar.gz')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import random
from datasets import Dataset, DatasetDict
from transformers import TrainingArguments
from trl import SFTTrainer
import os
from transformers import set_seed as transformers_set_seed

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = r"\llama3\Llama-3-8b-Instruct"

HAS_BFLOAT16 = torch.cuda.is_bf16_supported()
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.float16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)



source_train_file_ko_en = "ko_en.txt"
target_train_file_ko_en = "en_ko.txt"
source_train_file_vi_en = "vi_en.txt"
target_train_file_vi_en = "en_vi.txt"
source_train_file_vi_ko = "vi_ko.txt"
target_train_file_vi_ko = "ko_vi.txt"

with open(source_train_file_ko_en, encoding="utf-8") as source0, open(target_train_file_ko_en, encoding="utf-8") as target0:
  source_sentences0 = [sent0.strip() for sent0 in source0.readlines()]
  target_sentences0 = [sent0.strip() for sent0 in target0.readlines()]

with open(source_train_file_vi_en, encoding="utf-8") as source1, open(target_train_file_vi_en, encoding="utf-8") as target1:
  source_sentences1 = [sent1.strip() for sent1 in source1.readlines()]
  target_sentences1 = [sent1.strip() for sent1 in target1.readlines()]

with open(source_train_file_vi_ko, encoding="utf-8") as source2, open(target_train_file_vi_ko, encoding="utf-8") as target2:
  source_sentences2 = [sent2.strip() for sent2 in source2.readlines()]
  target_sentences2 = [sent2.strip() for sent2 in target2.readlines()]

def create_prompt():
    prompts = []
    #for i in range(0, len(source_sentences)):
    for i in range(0, 100):
      prompt_koen = "[INST]"+" Translate the Korean input text into English: " + source_sentences0[i] + " [/INST] " + target_sentences0[i] + " </s>"
      prompt_vien = "[INST]"+" Translate the Vietnamese input text into English: " + source_sentences1[i] + " [/INST] " + target_sentences1[i] + " </s>"
      prompt_enko = "[INST]"+" Translate the English input text into Korean: " + target_sentences0[i] + " [/INST] " + source_sentences0[i] + " </s>"
      prompt_envi = "[INST]"+" Translate the English input text into Vietnamese: " + target_sentences1[i] + " [/INST] " + source_sentences1[i] + " </s>"

      print(prompt_vien)
      prompts.append(prompt_koen)
      prompts.append(prompt_vien)
      prompts.append(prompt_enko)
      prompts.append(prompt_envi)
    for i in range(0,len(source_train_file_vi_ko)):
      prompt_kovi = "[INST]"+" Translate the Korean input text into Vietnamese: " + target_sentences2[i] + " [/INST] " + source_sentences2[i] + " </s>"
      prompt_viko = "[INST]"+" Translate the Vietnamese input text into Korean: " + source_sentences2[i] + " [/INST] " + target_sentences2[i] + " </s>"
      prompts.append(prompt_kovi)
      prompts.append(prompt_viko)
    return prompts

prompts = create_prompt()
random.shuffle(prompts)


dataset = DatasetDict({
    "train": Dataset.from_dict({"text": prompts[:len(prompts)]}),
    "validation": Dataset.from_dict({"text": prompts[int(0.9*len(prompts)):]})
})


random_state = 3407
transformers_set_seed(random_state)

output_directory = "llama3_20240711"

training_args = TrainingArguments(
                                  output_dir = output_directory,
                                  num_train_epochs=2,
                                  #max_steps = 594, # comment out this line if you want to train in epochs
                                  per_device_train_batch_size = 2,
                                  gradient_accumulation_steps = 1,
                                  optim="paged_adamw_8bit",
                                  fp16 = not HAS_BFLOAT16,
                                  bf16 = HAS_BFLOAT16,
                                  save_steps=1000,
                                  logging_steps=50,
                                  # save_strategy="epoch",
                                  evaluation_strategy=None,
                                  learning_rate=2e-4,
                                  weight_decay=0.001,
                                  max_grad_norm=0.3,
                                  max_steps=-1,
                                  warmup_ratio=0.03,
                                  group_by_length=True,
                                  lr_scheduler_type="linear",
                                  report_to=None,
                                  seed=random_state
                                )

max_seq_length = 1024  # increase if needed

trainer = SFTTrainer(
                    model=model,
                    # peft_config=lora_config,
                    max_seq_length=max_seq_length,
                    tokenizer=tokenizer,
                    packing=False,
                    dataset_text_field="text",
                    args=training_args,
                    train_dataset=dataset["train"],
                    eval_dataset=dataset["validation"],
                  )

trainer.train()

In [ ]:
!gdown --folder https://drive.google.com/drive/folders/sdsldkfj...somefileid.. -O /some_parent_directory/some_child_directory

In [ ]:
import gdown
url = 'https://drive.google.com/drive/folders/1aWSFb-ZR8TTIDdRlDBBCh-YvvCxmt6Bc'
output = '20150428_collected_images.tgz'
gdown.download(url, output, quiet=False)

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/drive/folders/1aWSFb-ZR8TTIDdRlDBBCh-YvvCxmt6Bc
To: /content/20150428_collected_images.tgz
1.21MB [00:00, 65.0MB/s]


'20150428_collected_images.tgz'

In [ ]:
!pip install datasets
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
   

In [ ]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset('werty1248/EnKo-Translation-LongTextOnly-dedup', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/161009 [00:00<?, ? examples/s]

In [ ]:
df = dataset.to_pandas()

In [ ]:
for index, row in df.head().iterrows():
  print(row['english'], '\n')

ROOFTOP GREENING STRUCTURETo provide a structure firmly and easily installing a house cultivation arch-like aggregate in a rooftop greening structure. This rooftop greening structure includes pressingly fixing each of support stands 24 in each of support stand line groups 24A to a rooftop slab surface through a greening support layer 6, using a fastener which pierces into the greening support layer 6, and steps over between each of the support stands 24 and the rooftop slab surface 2, and installing a holding member 36 for holding a house cultivation arch-like aggregate 50 each on the upper end surface of each of the support stands 24 in each of the support stand line groups 24A. As a result of this, the support stand 24 which has stiffness higher than the greening support layer 6, and is firmly fixed to the rooftop slab surface 2 through the greening support layer 6 is used for holding the end part of the arch-like aggregate 50. The holding member 36 for holding the end part of the ar

In [ ]:
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.utils.data
from torch import nn
from torchvision import models
import sys, os
# print(os.getcwd())
sys.path.append('./src')
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import patchcore
from patchcore import patchcore as patchcore_model
from siamese import SiameseNetwork
from torch.utils.data import TensorDataset, ConcatDataset, DataLoader
from torchvision import transforms
from typing import Callable, Any
import glob, os
import random

class Custom_Dataset_All(Dataset):
    """
    URL = https://www.robots.ox.ac.uk/~vgg/data/pets/
    """
    def __init__(self,
                 image_folder: str = r"D:\Non_Documents\9.Document\Dataset\mvtech_anomaly_detection\mvtech_anomaly_detection",
                 image_name: str = "bottle",
                 split: str = "train",
                 transform: Callable = None,
                **kwargs):
        self._sub_dataset = ('bottle',  'cable',  'capsule',  'carpet',  'grid', 'hazelnut', 'leather',  'metal_nut',  'pill', 'screw', 'tile', 'toothbrush', 'transistor', 'wood', 'zipper')
        self.image_name = image_name
        self.img_folder = image_folder
        self.transforms = transform
        self.imgs_pos, self.imgs_neg = self.get_image_paths(img_folder=image_folder)

    def get_image_paths(self, img_folder, split: str = "train"):
        img_positive = []
        img_negative = []
        # Define the file extensions for images
        image_extensions = ('*.png', '*.jpg', '*.jpeg', '*.gif', '*.bmp', '*.tiff')
        # Initialize an empty list to store image paths
        for img_name in self._sub_dataset:
            for ext in image_extensions:
                img_fold = os.path.join(img_folder, img_name, "train", "good")
                if (img_name == self.image_name):
                    img_positive.extend(glob.glob(os.path.join(img_fold, ext)))
                else:
                    img_negative.extend(glob.glob(os.path.join(img_fold, ext)))

        img_positive = img_positive[:int(len(img_positive) * 0.75)] if split == "train" else img_positive[int(len(img_positive) * 0.75):]
        img_positive = [self.pil_loader(x) for x in img_positive]
        img_negative = [self.pil_loader(x) for x in img_negative]
        random.shuffle(img_positive)
        random.shuffle(img_negative)
        return img_positive, img_negative

    def __len__(self):
        return len(self.imgs_pos)

    def pil_loader(self, path: str) -> Image.Image:
        # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
        with open(path, "rb") as f:
            img = Image.open(f).convert("RGB")

            if self.transforms is not None:
                img = self.transforms(img)
            return img

    def default_loader(self, path: str) -> Any:
            return self.pil_loader(path)

    def __getitem__(self, idx):
        valid_indices = [i for i in range(len(self.imgs_pos)) if i != idx]
        random_index = random.choice(valid_indices)
        img_anchor = self.imgs_pos[idx]
        img_pos = self.imgs_pos[random_index]
        img_neg = self.imgs_neg[random.randint(0, len(self.imgs_neg) - 1)]
        return img_anchor, img_pos,  img_neg

def _standard_patchcore(image_dimension):
    patchcore_instance = patchcore_model.PatchCore(torch.device("cpu"))
    backbone = models.wide_resnet50_2(pretrained=False)
    backbone.name, backbone.seed = "wideresnet50", 0
    patchcore_instance.load(
        backbone=backbone,
        layers_to_extract_from=["layer2", "layer3"],
        device=torch.device("cpu"),
        input_shape=[3, image_dimension, image_dimension],
        pretrain_embed_dimension=1024,
        target_embed_dimension=1024,
        patchsize=3,
        patchstride=1,
        spade_nn=2,
        sub_model=None
    )
    return patchcore_instance

if __name__ == "__main__":
    epoch_num = 20
    batch_size = 4
    IMAGENET_MEAN = [0.485, 0.456, 0.406]
    IMAGENET_STD = [0.229, 0.224, 0.225]
    network = SiameseNetwork(in_dimension=3).to(device='cuda')
    criterion = nn.TripletMarginLoss()
    criterion_2 = nn.MSELoss()
    optimizer = torch.optim.Adam(network.parameters(), lr=0.001)
    transfomation = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            # transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),

        ])
    _dataset = Custom_Dataset_All(transform=transfomation)
    _data_loader = DataLoader(_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

    _dataset_test = Custom_Dataset_All(transform=transfomation, split="test")
    _test_data_loader = DataLoader(_dataset_test, batch_size=batch_size, shuffle=True, drop_last=True)
    mean, std = 0, 0.2
    # model = _standard_patchcore(224)
    gaussian_noise = torch.randn(size=(batch_size, 3, 224, 224), device='cuda') * std + mean
    for epoch in range(epoch_num):
        print(f"=========== epoch {epoch} ===========")
        loss_item = 0.
        valid_loss = 0.
        network.train()
        for i, image in enumerate(_data_loader):
            if (image[0].shape[0] < batch_size): continue
            # image = torch.concat(image, dim=0)
            # feature = model.get_feature(image)
            bs, c, h, w = image[0].shape
            img0, img1, img2 = image[0], image[1], image[0].cuda() + gaussian_noise
            img0, img1, img2 = img0.cuda(), img1.cuda(), img2
            img2.requires_grad = True

            optimizer.zero_grad()
            output1, output2, output3 = network(img0, img1, img2)
            loss = criterion(output1, output2, output3) + criterion_2(img0, img2)

            loss.backward()
            optimizer.step()
            with torch.no_grad():
                xs = img2.grad
                gaussian_noise -= 0.001 * img2.grad
            loss_item += loss.item()
            if i % 1 == 0:
                print(f"Epoch [{epoch+1}/{epoch_num}], Step [{i}/{len(_data_loader)}], Loss: {(loss_item/(i+1)):.4f}")


        network.eval()
        with torch.no_grad():
            for i, image in enumerate(_test_data_loader):
                if (image[0].shape[0] < 2): continue
                img0, img1, img2 = image[0], image[1], image[0].cuda() + gaussian_noise
                img0, img1, img2 = img0.cuda(), img1.cuda(), img2

                output1, output2, output3 = network(img0, img1, img2)
                loss = criterion(output1, output2, output3) + criterion_2(img0, img2)
                valid_loss += loss.item()
            print(f"Epoch [{epoch+1}/{epoch_num}], Valid Loss: {(valid_loss/(len(_data_loader)+1)):.4f}")


    torch.save(network.state_dict(), f'./sub_model/siamese_wide_resnet50_2_final.pth')

In [ ]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
from custom_dataset import AudioDataset
from transformers import AutoFeatureExtractor
from datasets import Dataset, DatasetDict
from atc_audio import Audio_Implement
import numpy as np
import evaluate
accuracy = evaluate.load("accuracy")


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["file"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16_000*2, truncation=True
    )
    return inputs

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)


if __name__ == "__main__":
    model_path = './model_1'
    # model_path = './model_tiny/New folder'

    # model_path = './my_trained_model_tiny_cls/checkpoint-1782'
    dataset = AudioDataset(root_dir="./data", class_dir="./data/Voice_Class.txt")


    model = AutoModelForAudioClassification.from_pretrained(
        model_path, num_labels=len(dataset.class_bank), label2id=dataset.class_bank, id2label=dataset.class_bank_reverse, ignore_mismatched_sizes=True)

    print("class number: {}".format(len(dataset.class_bank)))
    print(dataset.class_bank)

    feature_extractor = AutoFeatureExtractor.from_pretrained(model_path)

    data_train = Dataset.from_dict(dataset.dataset_train_source)
    Audio_Implement.find_all_homophones()
    dataset_train = data_train.cast_column("file", Audio_Implement(sampling_rate=16_000))

    data_test = Dataset.from_dict(dataset.dataset_test_source)
    dataset_test = data_test.cast_column("file", Audio_Implement(sampling_rate=16_000))

    dataset_dict = DatasetDict({"train": dataset_train, "test": dataset_test})

    encoded_minds = dataset_dict.map(preprocess_function, remove_columns="file", batched=True)
    # encoded_minds = encoded_minds.rename_column("intent_class", "label")

    training_args = TrainingArguments(
        output_dir="my_trained_model_tiny_cls_29_c",
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=32,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=32,
        num_train_epochs=50,
        warmup_ratio=0.1,
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        push_to_hub=False,
        report_to='none',
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=encoded_minds["train"].with_format('torch'),
        eval_dataset=encoded_minds["test"].with_format('torch'),
        tokenizer=feature_extractor,
        compute_metrics=compute_metrics,
    )
    trainer.train()


In [ ]:
from torch import nn
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
from custom_dataset import AudioDataset
from transformers import AutoFeatureExtractor
from datasets import Dataset, DatasetDict, Audio
from network import Wave_Network
from torch.utils.data import DataLoader
import evaluate
accuracy = evaluate.load("accuracy")
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
# class Wave_Model(nn.Module):
#     def __init__(self, *args, **kwargs) -> None:
#         super().__init__(*args, **kwargs)
def collate_fn(batch):
    # Sử dụng pad_sequence để padding tất cả các chuỗi đến chiều dài lớn nhất trong batch
    input_ids = [torch.Tensor(item['input_values']) for item in batch]
    labels = [item['label'] for item in batch]

    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
    # for i in range(len(batch)):
    #     batch[i]['input_values'] = input_ids[i]
    # batch = pad_sequence(batch, batch_first=True, padding_value=0)
    return {
        'input_values': input_ids,
        'label': torch.tensor(labels)
    }

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["file"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16_000*2, truncation=True,
    )
    return inputs

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)


if __name__ == "__main__":
    # model_path = './model_1'
    model_path = './model_1'
    epoch_num = 100
    batch_size = 32
    dataset = AudioDataset(root_dir="./data", class_dir="./data/Voice_Class_anomaly.txt")

    model = Wave_Network(num_classes=len(dataset.class_bank), model_path=model_path).to(device='cuda')
    feature_extractor = AutoFeatureExtractor.from_pretrained(model_path)

    data_train = Dataset.from_dict(dataset.dataset_train_source)
    dataset_train = data_train.cast_column("file", Audio(sampling_rate=16_000))

    data_test = Dataset.from_dict(dataset.dataset_test_source)
    dataset_test = data_test.cast_column("file", Audio(sampling_rate=16_000))
    dataset_dict = DatasetDict({"train": dataset_train, "test": dataset_test})

    data_dict = dataset_dict.map(preprocess_function, remove_columns="file", batched=True)

    train_dataloader = DataLoader(data_dict['train'].with_format('torch'), batch_size, collate_fn=collate_fn)
    test_dataloader = DataLoader(data_dict['test'].with_format('torch'), batch_size, collate_fn=collate_fn)

    criterion_1 = nn.CrossEntropyLoss()
    criterion_2 = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.get_train_params(), lr=0.001)
    # optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epoch_num):
        loss_item = 0.
        valid_loss = 0.
        running_loss = 0
        save_loss = 999999
        model.train()
        for i, data in enumerate(train_dataloader):
            optimizer.zero_grad()
            inputs, labels = data['input_values'].cuda(), data['label'].cuda()
            true_project_label = torch.ones(size=(inputs.shape[0], 1))
            false_project_label = torch.zeros(size=(inputs.shape[0], 1))
            projection_labels = torch.concat((true_project_label, false_project_label)).cuda()
            o_nomaly, o_classification = model(inputs)
            loss = criterion_1(o_classification, labels) + criterion_2(o_nomaly, projection_labels)

            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 10 == 9:  # Print every 10 batches
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {loss.item()}')
                running_loss = 0.0

        model.eval()
        with torch.no_grad():
            for i, data in enumerate(test_dataloader):
                inputs, labels = data['input_values'].cuda(), data['label'].cuda()
                true_project_label = torch.ones(size=(inputs.shape[0], 1))
                false_project_label = torch.zeros(size=(inputs.shape[0], 1))
                projection_labels = torch.concat((true_project_label, false_project_label)).cuda()
                o_nomaly, o_classification = model(inputs)
                loss = criterion_1(o_classification, labels) + criterion_2(o_nomaly, projection_labels)
                valid_loss += loss.item()
            valid_loss = valid_loss/len(test_dataloader)
            if epoch == 0: valid_loss = save_loss
            else:
                if valid_loss < save_loss:
                    torch.save(model.state_dict(), f'./w2vec/models/w2vec_best.pth')
                    save_loss = valid_loss
            print(f"Epoch [{epoch+1}/{epoch_num}], Valid Loss: {valid_loss:.4f}")

    torch.save(model.state_dict(), f'./w2vec/models/w2vec_final.pth')


In [ ]:
from torch import nn
from transformers import Wav2Vec2Model, TrainingArguments, Trainer
from custom_dataset import AudioDataset
from transformers import AutoFeatureExtractor
from datasets import Dataset, DatasetDict
import torch
from torch import functional as F

class projection_MLP(nn.Module):
    def __init__(self, in_dim=256, hidden_dim=256, out_dim=256):
        super().__init__()
        ''' page 3 baseline setting
        Projection MLP. The projection MLP (in f) has BN ap-
        plied to each fully-connected (fc) layer, including its out-
        put fc. Its output fc has no ReLU. The hidden fc is 2048-d.
        This MLP has 3 layers.
        '''
        self.layer1 = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2)
        )
        self.layer2 = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2)
        )
        self.layer3 = nn.Sequential(
            nn.Linear(hidden_dim, out_dim),
            # nn.BatchNorm1d(out_dim)
        )
        self.num_layers = 3
    def set_layers(self, num_layers):
        self.num_layers = num_layers

    def forward(self, x):
        if self.num_layers == 3:
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
        elif self.num_layers == 2:
            x = self.layer1(x)
            x = self.layer3(x)
        else:
            raise Exception
        return x

class Wave_Network(nn.Module):
    def __init__(self, num_classes=18, model_path='', device='cuda'):
        super().__init__()
        self.device = device
        self.backbone = Wav2Vec2Model.from_pretrained(model_path).to(device=device)
        # self.backbone = nn.Sequential(*(list(self.backbone.children()))[:-2])

        for param in self.backbone.parameters():
            param.requires_grad = False

        self.linear = nn.Linear(in_features=768, out_features=256)
        # self.projection = projection_MLP().to(device=device)
        self.projection = nn.Sequential(
            nn.LazyLinear(256),
            nn.LazyLinear(256),
            nn.LeakyReLU(0.2)
        )
        self.tail = nn.Sequential(
                projection_MLP().to(device=device),
                nn.BatchNorm1d(256),
                nn.LeakyReLU(0.2),
                nn.LazyLinear(1, bias=False)
        )
        self.classifier = nn.LazyLinear(num_classes)

    def get_train_params(self):
        return [
            {'params': self.projection.parameters()},
            {'params': self.linear.parameters()},
            {'params': self.tail.parameters()},
            {'params': self.classifier.parameters()},
        ]
    def forward(self, x, std=0.05, is_train=True):
        with torch.no_grad():
            x = self.backbone(x).last_hidden_state
        x = self.linear(x)
        x_hidden_state = torch.mean(x, dim=1)
        # x = torch.max(x, dim=1).values
        # x = self.backbone(x).projector
        x_classification = self.classifier(x_hidden_state)

        x_projector = self.projection(x_hidden_state)
        # add noise
        if is_train:
            noise = torch.normal(mean=0, std=std, size=x_projector.shape).to(device=self.device)
            x_noise = x_projector + noise
            x_noise = torch.concat((x_projector, x_noise))
        else:
            x_noise = x_projector
        x_anomaly = self.tail(x_noise)

        return x_anomaly, x_classification



# model = Wave_Network(model_path='./model_1')
# print(model)


In [ ]:
from datasets import load_dataset, Audio
from transformers import pipeline
from custom_dataset import AudioDataset
from datasets import Dataset, Audio, DatasetDict
from network import Wave_Network
import torch
import torch.nn.functional as F

model_path = './model_1'
checkpoint_path = './W2Vec/models/w2vec_best.pth'
dataset = AudioDataset(root_dir=r"D:xxg\Voice\Label\labeling", class_dir="./data/Voice_Class_anomaly.txt", is_test=True)
model = Wave_Network(num_classes=len(dataset.class_bank), model_path=model_path, device='cpu')
model.load_state_dict(torch.load(checkpoint_path))
model = model.to(device='cpu')
model.eval()
data_train = Dataset.from_dict(dataset.dataset_test_source)
data_train = data_train.cast_column("file", Audio(sampling_rate=16_000))

count = 0
for i in range(len(data_train)):
    print(f'\npredicted result {data_train[i]["raw_path"]}:')
    audio_file = data_train[i]["file"]['array']
    input_tensor = torch.from_numpy(audio_file).unsqueeze(0).to(dtype=torch.float32, device='cpu')
    # padding = (0, 32000 - input_tensor.shape[-1])
    # input_tensor = F.pad(input_tensor, padding, "constant", 0)
    with torch.no_grad():
        o_nomaly, o_classification = model(input_tensor, is_train=False)
        print(torch.sigmoid(o_nomaly)[0].item(), dataset.data_bank[torch.argmax(o_classification, dim=-1)[0].item()])

In [ ]:
from typing import Dict
from datasets import Audio
import torch
import torchaudio
from typing import Dict, Optional, Union
import numpy as np
import random
import os, sys

sys.path.append(os.getcwd())
sys.path.append('./MeloTTS')
from MeloTTS.melo.api import TTS
tts_model = TTS(language="EN")


import nltk
import random

nltk.download('words')
from nltk.corpus import words
from nltk.corpus import cmudict
nltk.download('cmudict')
d = cmudict.dict()

import pronouncing
import random
import librosa


# List of words to avoid
avoid_phrases = ['Stand up', 'Sit down', 'Lie down', 'Go ahead', 'Move forward',
                            'Go straight', 'Go forward', 'Go backward', 'Reverse', 'Move backward',
                            'Go back', 'Stop', 'Go backward', 'Turn left', 'Turn right',
                            'Greeting', 'Love', 'Twist to the left', 'Twist to the right', 'Shake hand',
                            'Jump forward', 'Climbing mode', 'Stretch', 'Hi Lucy']
speaker_ids = tts_model.hps.data.spk2id
speakers = list(speaker_ids.keys())
# Get a list of all meaningful English words
english_words = set(words.words())
homophones_commands = []

class Audio_Implement(Audio):

    # def __init__(self, sampling_rate):
    #     super().__init__(sampling_rate=sampling_rate)

    def find_homophones(self, word_list):
        """Trả về danh sách các từ phát âm gần giống nhưng nghĩa khác"""
        homophones = {}
        for word in word_list:
            pronunciation = d.get(word.lower())
            if pronunciation:
                key = tuple(pronunciation[0])
                if key in homophones:
                    homophones[key].append(word)
                else:
                    homophones[key] = [word]
        return [words for words in homophones.values() if len(words) > 1]

    @staticmethod
    def find_all_homophones():
        for command in avoid_phrases:
            homophones = Audio_Implement.find_homophones(command)
            if homophones:
                random.shuffle(homophones)
                if len(homophones) > 100:
                    homophones_commands.extend(homophones[:100])
                else:
                    homophones_commands.extend(homophones)
        random.shuffle(homophones_commands)
    @staticmethod
    def find_homophones(command):

        words = command.split()
        homophone_sentences = []

        # Duyệt qua từng từ trong câu lệnh
        for i, word in enumerate(words):
            # Tìm tất cả các từ có phát âm giống với từ hiện tại
            pronunciations = pronouncing.phones_for_word(word)
            homophones = set()

            for pronunciation in pronunciations:
                homophones.update(pronouncing.search(pronunciation))

            homophones.discard(word)  # Bỏ từ gốc ra khỏi danh sách từ đồng âm
            for homophone in homophones:
                # Tạo câu lệnh mới với từ đồng âm
                new_sentence = words[:i] + [homophone] + words[i+1:]
                homophone_sentences.append(" ".join(new_sentence))
        return homophone_sentences

    # Function to generate random meaningful words that are not in the avoid list
    def generate_unique_meaningful_words(self, max_num_words=3):
        meaningful_words = list(english_words)
        # Filter out words that are part of the avoid phrases
        filtered_words = [word for word in meaningful_words if not any(phrase.lower().startswith(word) for phrase in avoid_phrases)]
        num_words = random.randint(1, max_num_words)
        gen = ' '.join(random.sample(filtered_words, num_words))
        # print("gend text: {}".format(gen))
        return gen

    def decode_example(self, value: dict, token_per_repo_id: Optional[Dict[str, Union[str, bool, None]]] = None) -> dict:
        snr_db = 10
        if value["path"] == "None":
            sample_rate = 16000  # 16 kHz
            duration_seconds = 2  # 5 seconds
            num_samples = sample_rate * duration_seconds
            random_audio_data = np.random.randn(num_samples)
            print("-----------------------------silence-----------------------------")
            return {"path": "None", "array": random_audio_data, "sampling_rate": 16000}

        elif value["path"] == "Unknown":
            words = self.generate_unique_meaningful_words(3)
            print(words)
            random_audio_data = tts_model.tts_to_file(words, speaker_ids[speakers[random.randint(0, len(speaker_ids) - 1)]], None, speed=1., noise_scale=0.1, noise_scale_w=0.2)
            random_audio_data = librosa.to_mono(random_audio_data)
            random_audio_data = librosa.resample(random_audio_data, orig_sr=44100, target_sr=16000)

            if random.randint(0, 1) == 1:
                noise = np.random.normal(0, 1, random_audio_data.shape[0])
                signal_power = np.mean(random_audio_data**2)
                noise_power = np.mean(noise**2)
                scaling_factor = np.sqrt(signal_power / (10**(snr_db / 10) * noise_power))
                scaled_noise = noise * scaling_factor
                random_audio_data = random_audio_data + scaled_noise
            return {"path": "unknown", "array": random_audio_data, "sampling_rate": 16000}

        else:
            data = super().decode_example(value, token_per_repo_id)
            if random.randint(0, 1) == 1:
                # noise = np.random.rand(data["array"].shape[0]) * 0.002
                noise = np.random.normal(0, 1, data["array"].shape[0])
                signal_power = np.mean(data["array"]**2)
                noise_power = np.mean(noise**2)
                scaling_factor = np.sqrt(signal_power / (10**(snr_db / 10) * noise_power))
                scaled_noise = noise * scaling_factor
                data["array"] = data["array"] + scaled_noise
            return data




















# class Audio_Implement(Audio):
#     def decode_example(self, value: dict, token_per_repo_id: Optional[Dict[str, Union[str, bool, None]]] = None) -> dict:
#         if value["path"] != "None":
#             data = super().decode_example(value, token_per_repo_id)
#             if random.randint(0, 1) == 1:
#                 noise = np.random.rand(data["array"].shape[0]) * 0.002
#                 data["array"] = data["array"] + noise
#             return data
#         else:
#             sample_rate = 16000  # 16 kHz
#             duration_seconds = 2  # 5 seconds
#             num_samples = sample_rate * duration_seconds
#             # random_audio_data = torch.randn(num_samples)
#             random_audio_data = np.random.rand(num_samples)
#             # random_audio_data = random_audio_data / torch.max(torch.abs(random_audio_data))
#             # audio_tensor = random_audio_data.unsqueeze(0)
#             # audio_bytes = torchaudio.functional.mu_law_encoding(audio_tensor, quantization_channels=256).squeeze().byte().numpy()
#             return {"path": "None", "array": random_audio_data, "sampling_rate": 16000}


In [ ]:
from transformers import AutoFeatureExtractor
from datasets import load_dataset, Audio
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
accuracy = evaluate.load("accuracy")


import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

dict_label = {'yes':0,
              'no':1,
              'up':2,
              'down':3,
              'left':4,
              'right':5,
              'on':6,
              'off':7,
              'stop':8,
              'go':9,
              'unknown':10,
              'silence':11}

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    y_pred = np.argmax(predictions, axis=1)

    # Calculate the confusion matrix
    cm = confusion_matrix(labels, y_pred)

    # Plot the confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

    # You can return additional metrics if needed
    accuracy = accuracy_score(labels, y_pred)
    return {"accuracy": accuracy}


# def compute_metrics(eval_pred):
#     predictions = np.argmax(eval_pred.predictions, axis=1)
#     return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16_000, truncation=True
    )
    return inputs




def edit_label(examples):
    # valid_dataset.features['label'].names = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', '_silence_']
    for i, x in enumerate(examples['file']):
        a = examples['label'][i]
        if examples['is_unknown'][i]: examples['label'][i] = dict_label['unknown']
        elif id2label[str(examples['label'][i])] == '_silence_':
            examples['label'][i] = dict_label['silence']
    return examples

feature_extractor = AutoFeatureExtractor.from_pretrained('./model_1')
train_dataset_ = load_dataset("google/speech_commands", 'v0.01', split='train', trust_remote_code=True)
valid_dataset_ = load_dataset("google/speech_commands", 'v0.01', split='validation', trust_remote_code=True)
test_dataset_ = load_dataset("google/speech_commands", 'v0.01', split='test', trust_remote_code=True)

labels = valid_dataset_.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label


train_dataset_ = train_dataset_.map(edit_label, batched=True)
valid_dataset_ = valid_dataset_.map(edit_label, batched=True)
test_dataset_ = test_dataset_.map(edit_label, batched=True)

df = train_dataset_.to_pandas()
# Giả sử cột label trong dataset có tên là 'label'
label_counts = df['label'].value_counts()

# Hiển thị kết quả
print(label_counts)

# # Hiển thị đồ thị
# label_counts.plot(kind='bar')
# plt.xlabel('Class')
# plt.ylabel('Số lượng')
# plt.title('Số lượng mỗi class trong cột label')
# plt.show()

train_dataset_ = train_dataset_.remove_columns(["file", "is_unknown", "speaker_id", "utterance_id"])
valid_dataset_ = valid_dataset_.remove_columns(["file", "is_unknown", "speaker_id", "utterance_id"])
test_dataset_ = test_dataset_.remove_columns(["file", "is_unknown", "speaker_id", "utterance_id"])

labels = dict_label.keys()
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
print(label2id)



train_data = train_dataset_.cast_column("audio", Audio(sampling_rate=16_000))
valid_data = valid_dataset_.cast_column("audio", Audio(sampling_rate=16_000))
test_dataset = test_dataset_.cast_column("audio", Audio(sampling_rate=16_000))

encoded_data_train = train_data.map(preprocess_function, remove_columns="audio", batched=True)
encoded_data_validation = valid_data.map(preprocess_function, remove_columns="audio", batched=True)
encoded_test_validation = test_dataset.map(preprocess_function, remove_columns="audio", batched=True)
print(encoded_data_train)

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    # "./model_1", num_labels=num_labels, label2id=label2id, id2label=id2label
    "./my_awesome_mind_model/checkpoint-3990", num_labels=num_labels, label2id=label2id, id2label=id2label
)
training_args = TrainingArguments(
    output_dir="my_awesome_mind_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    report_to='none',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_data_train.with_format("torch"),
    eval_dataset=encoded_data_validation.with_format("torch"),
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)
# trainer.train()

eval_results = trainer.evaluate(eval_dataset=encoded_test_validation.with_format("torch"))
print(eval_results)


In [ ]:
import ctranslate2
import transformers


# model = ctranslate2.Translator(r"D:\1.Project\6.ChatBot\convert\convert_m2m-9548", device="cuda", device_index=[0]) # file convert ra dc
model = ctranslate2.Translator(r"xxxconvert\convert_m2m") # file convert ra dc
tokenizer = transformers.AutoTokenizer.from_pretrained(r"xxx\convert\checkpoint-9548") # copy các file token ở model gốc ra

ct2-transformers-converter --model D:\1.Project\6.ChatBot\convert\checkpoint-23870 --output_dir D:\1.Project\6.ChatBot\convert\convert_m2m --force

src_lang = 'ko'
tgt_lang = 'vi'
tokenizer.src_lang = src_lang
batch_sentences = [
    "Quỹ đầu tư Hermes đã sử dụng dữ liệu từ khu vực châu Âu và có được những dự đoán riêng của mình để tính toán chính xác các nước đang phải đối mặt những khó khăn lớn nhất.",
]
# batch_tokens = [tokenizer.encode(sentence, )[0].tolist() for sentence in batch_sentences]
target_prefix = [[tokenizer.lang_code_to_token[tgt_lang]]] * len(batch_sentences)
source_sentences = [[tokenizer.lang_code_to_token[src_lang]] + tokenizer.tokenize(sentence) for sentence in batch_sentences]
input_tokens = []
for i in range(0, len(batch_sentences)):
    input_tokens.append(tokenizer.convert_ids_to_tokens(tokenizer.encode(batch_sentences[i])))
translations = model.translate_batch(input_tokens, target_prefix=target_prefix)
translations = [translation[0]['tokens'][1:] for translation in translations]
translations = [tokenizer.convert_tokens_to_string(translation) for translation in translations]
for translation in translations:
    print(translation)